In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg
#from delta import *

# Inicializar a sessão Spark
minio_endpoint = "http://minio:9000"
minio_access_key = "minio"
minio_secret_key = "minio123"

spark = SparkSession.builder \
    .appName("cdcproject") \
    .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) \
    .config("spark.hadoop.fs.s3a.access.key", minio_access_key) \
    .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

print("Sessão Spark inicializada com suporte ao Delta Lake.")

delta_table_path = "s3a://raw/ecommerce_db/postgres.public.orders*.json"
orders_df = spark.read.format("json").load(delta_table_path)

Sessão Spark inicializada com suporte ao Delta Lake.


In [5]:
orders_df.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- after: struct (nullable = true)
 |    |    |-- customer_id: string (nullable = true)
 |    |    |-- order_approved_at: long (nullable = true)
 |    |    |-- order_delivered_timestamp: long (nullable = true)
 |    |    |-- order_estimated_delivery_date: long (nullable = true)
 |    |    |-- order_id: string (nullable = true)
 |    |    |-- order_purchase_timestamp: long (nullable = true)
 |    |    |-- order_status: string (nullable = true)
 |    |-- before: string (nullable = true)
 |    |-- op: string (nullable = true)
 |    |-- source: struct (nullable = true)
 |    |    |-- connector: string (nullable = true)
 |    |    |-- db: string (nullable = true)
 |    |    |-- lsn: long (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- schema: string (nullable = true)
 |    |    |-- sequence: string (nullable = true)
 |    |    |-- snapshot: string (nullable = true)
 |    |    |-- table: string (nullable = true)


In [6]:
spark.stop()